```
ME72: Maestría en Métodos Cuantitativos para la Gestión y Análisis de Datos
M72109: Analisis de Datos no Estructurados
Universidad de Buenos Aires - Facultad de Ciencias Economicas (UBA-FCE)
Año: 2020
Profesor: Facundo Santiago
```

# Word2Vec - Modelos basados en secuencias

Los modelos basados en secuencias tienen la fortaleza que toman una secuencia de token (en un determinado orden) y generan una salida dependiendo del tipo de problema que se trate.
 - Seq2Class: Toman una secuencia de tokens y generan una clase
 - Seq2Seq: Toman una secuencia de token y generan otra secuencia de tokens. 
 
En este notebook exploraremos este tipo de modelos utilizandolos de la mano de word2vec:

## Prepración del ambiente

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Descarguemos algunos fragmentos de código para simplificar el trabajo

In [ ]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/TextNormalizer.py --directory-prefix ./Utils/
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/PadSequenceTransformer.py --directory-prefix ./Utils/
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/Word2VecVectorizer.py --directory-prefix ./Utils/

Descargamos el set de datos

In [ ]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv --directory-prefix ./Datasets/mascorpus/

Descargamos nuestros vectores de word2vec en español

In [ ]:
!mkdir -p ./Models/Word2Vec
!wget https://santiagxf.blob.core.windows.net/public/Word2Vec/model.bin --directory-prefix ./Models/Word2Vec

Instalamos las librerias necesarias

In [ ]:
!pip install unidecode
!python -m spacy download es_core_news_sm

Cargamos el set de datos

In [1]:
import pandas as pd

tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tweets['TEXTO'], tweets['SECTOR'], 
                                                    test_size=0.33, 
                                                    stratify=tweets['SECTOR'])

## Preprocesamiento de texto

Al igual que con Topic Modeling, nuestro primer paso es preprocesar el texto. Para focalizarnos en Word2Vec en este modulo, les preparé un modulo TweetTextNormalizer que hará todo el preprocesamiento por nosotros. Pueden explorar los parametros que recibe el constructor de esta clase para ver que opciones podemos configurar como Stemmer, Lemmatization, etc.

En lo particular, estamos creando un TweetTextNormalizer que:
 - Aplicará un tokenizer especifico para Twitter
 - Eliminará stop words
 - Aplicará lemmatization
 - Eliminará URLs
 - Eliminará acentos
 - Eliminará las mayusculas
 
Adicionalmente, el parametro text_to_sequence=True indica que la salida de este proceso no serán oraciones sino que tokens.

In [1]:
from Utils.TextNormalizer import TweetTextNormalizer

In [3]:
normalizer = TweetTextNormalizer(preserve_case=False,text_to_sequence=True)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Podemos probar como funciona:

## Vectorización de las palabras

En las actividades anteriores utilizamos siempre un TF-IDF vectorizer para generar los vectores. En esta oportunidad utilizaremos Word2Vec utilizando un modelo pre-entrenado para el idioma español. Adicionalmente, vemos que este vectorizer tiene el parametro sequence_to_idx en Verdadero. Esto significa que no queremos que como salida obtengamos los vectores de Word2Vec, sino que queremos "el indice" que se corresponde a la palabra en una matriz de indice-palabra/vectores. 

In [4]:
from Utils.Word2VecVectorizer import Word2VecVectorizer

In [5]:
w2v = Word2VecVectorizer(model_path='Models/Word2Vec/model.bin', sequence_to_idx=True)
embedding_weights = w2v.get_weights()

El método get_weights() construye la matríz de indice-palabra/vector que luego será utilizado para encontrar los vectores correspondientes de cada palabra. Esta matriz tiene dimensiones m x n, donde m es la cantidad de palabras del vocabulario y n la dimensión de los vectores de word2vec. En este caso trabajamos con vectores de dimensionalidad 100.

## Modelos basados en sequencias

Vimos que cuando aplicamos técnicas de Topic Modeling, intentamos reducir la cantidad de dimensiones de nuestras representaciones de palabras para luego utilizar un clasificador para resolver la tarea en cuestión. Sin embargo, la suponsición básica de ese tipo de modelos es que un texto no es mas que una distribución de palabras (bag of words). Sin embargo, nosotros sabemos que un texto es una secuencia de palabras donde importa el orden. Para capturar este tipo de propiedades podemos utilizar modelos basados en secuencias.

### Construirmos un modelo basado en secuencias

Para construir nuestro modelo, utilizaremos TensorFlow. En particular utilizaremos la API de Keras que nos permite componer modelos de redes neuronales como una secuencia de pasos o capas que se conectan en una dirección. 

Utilizemos los siguientes tipos de capas:
 *  **Embedding:** Esta capa transforma vectores que representan indices dentro de una matriz en representaciones vectoriales densas. Básicamente en este caso nos resolverá la busqueda de las representaciones vectoriales para nuestras palabras.
 * **SpatialDropout1D:** Este tipo de capas ayudan a promover la independencia entre filtros (feature maps). Funciona en forma analoga a Dropout pero en lugar de desconectar elementos individuales, desconecta el filtro completo.
 * **LSTM:** Long Short-Term Memory layer - Hochreiter 1997
 * **Dense:** Una típica capa de una red neuronal completamente conectada (fully connected)

Algunos detalles para notar:
 * *loss='sparse_categorical_crossentropy'*, este problema de clasificación (crossentropy) de más de una clase (categorical). Sin embargo, nuestro output produce probabilidades de cada una de las clases posibles (7) en forma one-hot encoding. 
 * *metrics=['accuracy']*: Si bien nuestra metrica es accuracy, Keras hará un promedio ponderado del accuracy de cada clase. Este es el comportamiento por defecto.

In [9]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, SpatialDropout1D

In [10]:
def build_model(sequence_len, vocab_size, emdedding_size, embedding_weights):
    model = Sequential([
        Embedding(vocab_size, emdedding_size,
                  weights=[embedding_weights],
                  trainable=False,
                  mask_zero=True),
        SpatialDropout1D(0.2),
        LSTM(emdedding_size),
        Dense(7, activation='softmax')
    ])
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Podemos inspeccionar el modelo:

Para continuar utilizando la misma metodología que veniamos utilizando antes, embeberemos este clasificador diseñado en TensorFlow dentro de un pipeline en Scikit-Learn. Esto lo podemos lograr utilizando KerasClassifier. Este tipo de elementos se los conoce como Wrappers. Este paso no es necesario siempre, en nuestro caso solo porque estamos trabajando con Scikit-Learn como nuestro orquestador:

In [11]:
estimator = keras.wrappers.scikit_learn.KerasClassifier(
    build_fn=build_model, 
    epochs=50,
    sequence_len=max_seq_len,
    vocab_size=w2v.vocab_size,
    emdedding_size=w2v.emdedding_size,
    embedding_weights=embedding_weights)

### Ajustando la longitud de las secuencias

Los modelos basados en secuencias pueden adaptarse a cualquier longitud de secuencia, sin embargo, los parametros de nuestras redes neuronales deberan ser fijos. Para esto definiermos una longitud máxima de la secuencia que vamos analizar. Para esto podemos utilizar un valor especifico o utilizar el valor máximo de tokens que hay en nuestro corpus.

La siguiente clase PadSequenceTransformer es un modulo que les preparé para simplificar este procesamiento. El mismo se encarga de ajustar cualquier secuencia para que tenga exactamente max_seq_len. Cuando la lingitud es mejor, se completan con ceros.

In [6]:
from Utils.PadSequenceTransformer import PadSequenceTransformer

In [7]:
max_seq_len = 100

In [8]:
seq2seq = PadSequenceTransformer(max_len=max_seq_len)

## Creando nuestro pipeline

Construimos nuestro pipeline como siempre combinando los diferentes pasos:

In [12]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(steps=[('normalizer', normalizer), 
                           ('vectorizer', w2v),
                           ('padder', seq2seq),
                           ('estimator', estimator)])

Entrenamos nuestro modelo

In [14]:
model = pipeline.fit(X=X_train, y=y_train)

Epoch 1/50
79/79 [==============================] - 14s 179ms/step - loss: 1.7382 - accuracy: 0.3392
Epoch 2/50
79/79 [==============================] - 14s 176ms/step - loss: 1.4215 - accuracy: 0.4891
Epoch 3/50
79/79 [==============================] - 14s 178ms/step - loss: 1.2835 - accuracy: 0.5363
Epoch 4/50
79/79 [==============================] - 14s 175ms/step - loss: 1.2324 - accuracy: 0.5498
Epoch 5/50
79/79 [==============================] - 14s 178ms/step - loss: 1.1720 - accuracy: 0.5748
Epoch 6/50
79/79 [==============================] - 14s 177ms/step - loss: 1.1465 - accuracy: 0.5791
Epoch 7/50
79/79 [==============================] - 14s 173ms/step - loss: 1.1214 - accuracy: 0.5867
Epoch 8/50
79/79 [==============================] - 14s 174ms/step - loss: 1.0836 - accuracy: 0.6002
Epoch 9/50
79/79 [==============================] - 14s 177ms/step - loss: 1.0687 - accuracy: 0.5974
Epoch 10/50
79/79 [==============================] - 14s 177ms/step - loss: 1.0422 - accura

## Evalución de los resultados

Probamos su performance utilizando el test set

In [15]:
predictions = model.predict(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [16]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

ALIMENTACION       0.78      0.64      0.70       110
  AUTOMOCION       0.50      0.51      0.50       148
       BANCA       0.57      0.51      0.54       198
     BEBIDAS       0.58      0.72      0.64       223
    DEPORTES       0.49      0.63      0.55       216
      RETAIL       0.61      0.52      0.56       268
       TELCO       0.62      0.38      0.47        79

    accuracy                           0.57      1242
   macro avg       0.59      0.56      0.57      1242
weighted avg       0.58      0.57      0.57      1242

